In [52]:
#Importing the modules and packages to be used in this project
import numpy as np
import sympy as sm
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interactive
plt.style.use('seaborn-whitegrid')
from scipy import optimize

import model_project as mp
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In this model project we want to solve the Solow model with human capital. We take the model from 'Introducing Advanced Macroeconomics - Growth and Business Cycles'. 3 edition 2022, Sørensen, Peter Birch and Whitta-Jacobsen, Hans Jørgen.

The Solow model with human capital from Mankiw, Weil and Romer is an extenstion of the general Solow model, that helps explain why international investments does not flow to poorer countries as predicted by the general Solow model. The modification finds itself in the addition of human capital, which is mostly measured as years of education and experience in general, and sheds greater light on why some countries are rich and some are poor. 

# The baseline model

In [2]:
#First we define all the parameters and variables using sympy
Y, K, alpha, H, phi, A, L, y, r, w, n, g, K1, sk, sh, H1, delta, L1, A1, tau, h_tilde, h_tilde1, k_tilde, k_tilde1,\
k_tilde_star, h_tilde_star, y_star, Y_ex, K_ex, alpha_ex, H_ex, phi_ex, A_ex, L_ex, y_ex, n_ex, g_ex, K1_ex, sk_ex,\
sh_ex, H1_ex, delta_ex, L1_ex, A1_ex, tau_ex, h_tilde_ex, h_tilde1_ex, k_tilde_ex, k_tilde1_ex, k_tilde_star_ex,\
h_tilde_star_ex, y_star_ex = sm.symbols('Y_t, K_t, alpha, H_t, phi, A_t, L_t, y_t, r_t, w_t, n, g, K_{t+1}, s_K, s_H,\
H_{t+1}, delta, L_{t+1}, A_{t+1}, tau, htilde_t, htilde_t+1, ktilde_t, ktilde_t+1, ktilde^*, htilde^*, y^*, Y_t, K_t,\
alpha, H_t, phi, A_t, L_t, y_t, n, g, K_{t+1}, s_K, s_H, H_{t+1}, delta, L_{t+1}, A_{t+1}, tau, htilde_t,\
htilde_t+1, ktilde_t, ktilde_t+1, ktilde^*, htilde^*, y^*') 

In [3]:
#Here we create all the equations of the model and displays them again using sympy
Y_t = sm.Eq(Y, (K**alpha*H**phi)*(A*L)**(1-alpha-phi))
r_t = sm.Eq(r, alpha*(K/(A*L))**(alpha-1)*(H/(A*L))**phi)
w_t = sm.Eq(w, (1-alpha)*(K/(A*L))**alpha*(H/(A*L))**phi*A)
K_t1 = sm.Eq(K1, sk*Y+(1-delta)*K)
H_t1 = sm.Eq(H1, sh*Y+(1-delta)*H)
L_t1 = sm.Eq(L1, (1+n)*L)
A_t1 = sm.Eq(A1, (1+g)*A)
display(Y_t, r_t, w_t, K_t1, H_t1, L_t1, A_t1)

Eq(Y_t, H_t**phi*K_t**alpha*(A_t*L_t)**(-alpha - phi + 1))

Eq(r_t, alpha*(H_t/(A_t*L_t))**phi*(K_t/(A_t*L_t))**(alpha - 1))

Eq(w_t, A_t*(H_t/(A_t*L_t))**phi*(K_t/(A_t*L_t))**alpha*(1 - alpha))

Eq(K_{t+1}, K_t*(1 - delta) + Y_t*s_K)

Eq(H_{t+1}, H_t*(1 - delta) + Y_t*s_H)

Eq(L_{t+1}, L_t*(n + 1))

Eq(A_{t+1}, A_t*(g + 1))

The Solow model with human capital is given with these equations. 1) stating that output is given as a Cobb-Douglas production function. 2 and 3) stating that the real rental rate on capital and real wage is given from the marginalproduct of capital and labour, respectively. 4 and 5) stating the accumulation equation of capital and human capital. 6) stating that labour grows with the exogeneously rate of n. 7) stating that technology grows with the exogeneously rate of g.

# Solving the model analytically 

First we would like to solve the model symbolically with sympy meaning that we define the transition- and Solow equations to thereafter define the steady state values. We begin by deriving the transition equation and thereafter we define the Solow eqautions.

The law of motion in the Solow model with human capital can be descibed by the transition- and Solow equations. The transition equation and Solow equations states how all of the parameters in the model move over time and is stated below.

In [4]:
k_trans = sm.Eq(k_tilde1, ((1/(1+n)*(1+g))*(sk*(k_tilde**alpha)*(h_tilde**phi)+(1-delta)*k_tilde)))
h_trans = sm.Eq(h_tilde1, ((1/(1+n)*(1+g))*(sh*(k_tilde**alpha)*(h_tilde**phi)+(1-delta)*h_tilde)))
display(k_trans, h_trans)

Eq(ktilde_t+1, (g + 1)*(htilde_t**phi*ktilde_t**alpha*s_K + ktilde_t*(1 - delta))/(n + 1))

Eq(htilde_t+1, (g + 1)*(htilde_t*(1 - delta) + htilde_t**phi*ktilde_t**alpha*s_H)/(n + 1))

The Solow eqautions can be found by subtracting both sides of the equation by $\tilde{k}_t$ and $\tilde{h}_t$, respectively. The Solow eqautions therefore become.

In [5]:
k_solow = sm.Eq(k_tilde1-k_tilde, ((1/(1+n)*(1+g))*(sk*(k_tilde**alpha)*(h_tilde**phi)-(n+g+delta+n*g)*k_tilde)))
h_solow = sm.Eq(h_tilde1-h_tilde, ((1/(1+n)*(1+g))*(sh*(k_tilde**alpha)*(h_tilde**phi)-(n+g+delta+n*g)*k_tilde)))
display(k_solow, h_solow)

Eq(-ktilde_t + ktilde_t+1, (g + 1)*(htilde_t**phi*ktilde_t**alpha*s_K - ktilde_t*(delta + g*n + g + n))/(n + 1))

Eq(-htilde_t + htilde_t+1, (g + 1)*(htilde_t**phi*ktilde_t**alpha*s_H - ktilde_t*(delta + g*n + g + n))/(n + 1))

As we know analytically these equations can be solved to find steady state by setting $\tilde{k}_{t+1}$ and $\tilde{k}_{t}$ equal to eachother and solving for $\tilde{k}_{t}$ and the same for $\tilde{h}_{t+1}$ and $\tilde{h}_{t}$. That is the same as solving the right hand side and the left hand side in the big parenthesis to be equal to eachother. If this is done we end up with the following steady state values:

In [6]:
k_steady_state = sm.Eq(k_tilde_star, ((sk**(1-phi)*(sh**phi))/(n+g+delta+n*g))**(1/(1-alpha-phi)))
h_steady_state = sm.Eq(h_tilde_star, ((sk**(alpha)*(sh**(1-alpha)))/(n+g+delta+n*g))**(1/(1-alpha-phi)))
print('So physical capital and human capital is in steady state equal to:')
display(k_steady_state, h_steady_state)

So physical capital and human capital is in steady state equal to:


Eq(ktilde^*, (s_H**phi*s_K**(1 - phi)/(delta + g*n + g + n))**(1/(-alpha - phi + 1)))

Eq(htilde^*, (s_H**(1 - alpha)*s_K**alpha/(delta + g*n + g + n))**(1/(-alpha - phi + 1)))

We can now find ${y^*}$ out of our definition on $\tilde{y^{*}}$ = $\frac{y^{*}}{A_{{t}}}$  <=> ${y^*}$ = ${A}_{t}*\tilde{k^\alpha}_{t}*\tilde{h^\phi}_{t}$

In [7]:
sk_div = ((sk/(n+g+delta+n*g))**(alpha/(1-alpha-phi))) #For more cleaner code

y_steady_state = sm.Eq(y_star, (sk_div*(sh/(n+g+delta+n*g))**(phi/(1-alpha-phi)))*A)
print('GDP per worker is in steady state equal to')
display(y_steady_state)

GDP per worker is in steady state equal to


Eq(y^*, A_t*(s_H/(delta + g*n + g + n))**(phi/(-alpha - phi + 1))*(s_K/(delta + g*n + g + n))**(alpha/(-alpha - phi + 1)))

Now that we have all of our steady state values we can begin to calculate them by sympy for given parameter values.
From Weil, Mankiw and Romer (taking from the book) we get the following 'realistic' parameter values for all of our exogenous variables.

In [8]:
delta = 0.06
n = 0
g = 0.015
alpha = (1/3)
phi = (1/3)
sk = 0.2
sh = 0.15
tau = 0.15
A = 1 #Most estimates set A to be around 1, so therefore the value that it has

Now all there is left to do is to restate the equations, but now with the parameter values as given from above.

In [9]:
sk_div2 = ((sk/(n+g+delta+n*g))**(alpha/(1-alpha-phi))) #For more cleaner code

k_steady_state_value = sm.Eq(k_tilde_star, (((sk**(1-phi))*(sh**phi))/(n+g+delta+n*g))**(1/(1-alpha-phi)))
h_steady_state_value = sm.Eq(h_tilde_star, (((sk**alpha)*(sh**(1-alpha)))/(n+g+delta+n*g))**(1/(1-alpha-phi)))
y_steady_state_value = sm.Eq(y_star, (((sk_div2*((sh/(n+g+delta+n*g))**(phi/(1-alpha-phi))))*A)))
print('With the given parameter values. Physical capital per effective worker is in steady state:')
display(k_steady_state_value)
print('With the given parameter values. Human capital per effective worker is in steady state:')
display(h_steady_state_value)
print('With the given parameter values. GDP per worker is in steady state:')
display(y_steady_state_value)

With the given parameter values. Physical capital per effective worker is in steady state:


Eq(ktilde^*, 14.2222222222222)

With the given parameter values. Human capital per effective worker is in steady state:


Eq(htilde^*, 10.6666666666667)

With the given parameter values. GDP per worker is in steady state:


Eq(y^*, 5.33333333333333)

Not so weirdly we get a high value of $\tilde{k^*}$ because of the savings rate on physical capital is higher than on human capital. we do however in the expressions see something odd that is both human capital and physical capital depends on both the savings rate of human capital and the savings rate of physical capital. We find the explaination to that in something called self-reinforcing- and cross effects. The effects can be seen in that a higher savings rate on human capital will infest itself in a higher GDP per (effective) worker which will in itself bring on more savings to both human- and physical capital (cross effect) which will further root itself in higher levels of physical- and human capital and so on(self-reinforcing effects). This will continue for infinty, though the effect will at last become almost equal to 0. That is why we see such a high level of GDP per worker in steady state.  

# Interactive phase diagram

In this section we visualize the transition to steady state via an interactive phase diagram.

To be able to draw the phase diagram we must first have the so called nullclines.
The nullclines can be isolated from the Solow eqautions as follows

In [10]:
#First we set the Solow equations equal to 0
null_k = sm.Eq(0, ((sk_ex*(k_tilde**alpha_ex)*(h_tilde**phi_ex)-(n_ex+g_ex+delta_ex+n_ex*g_ex)*k_tilde)))
null_h = sm.Eq(0, ((sh_ex*(k_tilde**alpha_ex)*(h_tilde**phi_ex)-(n_ex+g_ex+delta_ex+n_ex*g_ex)*h_tilde)))
display(null_k, null_h)

Eq(0, htilde_t**phi*ktilde_t**alpha*s_K - ktilde_t*(delta + g*n + g + n))

Eq(0, -htilde_t*(delta + g*n + g + n) + htilde_t**phi*ktilde_t**alpha*s_H)

Now that we have these we can solve for $\tilde{h}_t$ in both of these equations and we will then get the nullclines

In [11]:
nullcline_k = sm.solve(null_k, h_tilde)[0]
nullcline_h = sm.solve(null_h, h_tilde)[0]
display(nullcline_k, nullcline_h)

(ktilde_t**(1 - alpha)*(delta + g*n + g + n)/s_K)**(1/phi)

(ktilde_t**alpha*s_H/(delta + g*n + g + n))**(-1/(phi - 1))

These are our so-called nullclines for $\tilde{k}_t$ and $\tilde{h}_t$, respectively.
It is not so hard to see with realistic parameter values of $\phi$=$\alpha$=1/3 that $\tilde{k}_t$ is a $\tilde{k}^2$ function and that $\tilde{h}$ is a $\tilde{h}^{1/2}$ function, nevertheles we show down below the transition to steady state.

In [72]:
#a. First we define the two nullclines as functions
def nullcline2_k(k, alpha, sk, phi, n, g, delta):
    return ((((k**(1-alpha))*(delta+n+g+n*g))/sk)**(1/phi))

def nullcline2_h(k, alpha, sh, phi, n, g, delta):
    return ((((k**alpha)*sh)/(delta+n+g+n*g))**(-1/(phi-1)))

#b. Now we can define the interactive figure
def interac_phase(n, g, delta, alpha, phi, sk, sh):

    #Restrictions on the model
    if alpha+phi >= 1 and sh+sk >= 1:
        result = print('$\alpha+\varphi$ and $s_H+s_K$ have to be stricly smaller than 1')
    elif sh+sk >= 1:
        result = print('$s_H+s_K$ has to be stricly smaller than 1')
    elif alpha+phi >= 1:
        result = print('$\alpha+\varphi$ has to be stricly smaller than 1')
    elif n+g+delta+n*g <= 0:
        result = print('$n+g+\delta+ng$ has to be strictly greater than $0$')
    
    #Range for k
    k = np.arange(0, 100, 0.1)
    
    #Figure size in inches
    plt.figure(dpi=120)
    
    plt.xlabel(r'$\tilde{k}_t$',fontsize=10, color = 'black' )
    plt.ylabel(r'$\tilde{h}_t$', fontsize=10, color = 'black')
    plt.title('Figure 1: The Solow diagram with human capital', fontsize=10)
    
    #Plotting the nullclines
    plt.plot(k, nullcline2_k(k, sk, phi, alpha, delta, n, g))
    plt.plot(k, nullcline2_h(k, sh, alpha, phi, delta, n, g))

    #Plotting the legend
    plt.legend((r'$\left[\Delta\tilde{k}_{t}=0\right]$',
                    r'$\left[\Delta\tilde{h}_{t}=0\right]$'),
                   shadow=True, handlelength=1.4, fontsize=10)
    
    plt.axis([0,20,0,20])
    plt.show()

#Plotting the figure and interactive slider for the parameter values
interactive_plot = interactive(interac_phase, 
                               sk=widgets.FloatSlider(description='$s_K$', 
                                                       min=0.1,
                                                       max=0.99,
                                                       step=0.05,
                                                       value=0.20,
                                                       continuous_update=True),
                               sh=widgets.FloatSlider(description='$s_H$', 
                                                       min=0.1,
                                                       max=0.99,
                                                       step=0.05,
                                                       value=0.15,
                                                       continuous_update=True),
                               alpha=widgets.FloatSlider(description='$\alpha$',
                                                         min=0.01,
                                                         max=0.99,
                                                         step=0.05,
                                                         value=(1/3),
                                                         continuous_update=True),
                               phi=widgets.FloatSlider(description='$\phi$',
                                                         min=0.01,
                                                         max=0.99,
                                                         step=0.05,
                                                         value=(1/3),
                                                         continuous_update=True),
                               n=widgets.FloatSlider(description='$n$',
                                                         min=-0.99,
                                                         max=0.99,
                                                         step=0.05,
                                                         value=0,
                                                         continuous_update=True),
                               g=widgets.FloatSlider(description='$g$',
                                                         min=-0.99,
                                                         max=0.99,
                                                         step=0.05,
                                                         value=0.15,
                                                         continuous_update=True),
                               delta=widgets.FloatSlider(description='$\delta$',
                                                         min=0.01,
                                                         max=0.99,
                                                         step=0.05,
                                                         value=0.06,
                                                         continuous_update=True)
                              )
                              
interactive_plot

interactive(children=(FloatSlider(value=0.0, description='$n$', max=0.99, min=-0.99, step=0.05), FloatSlider(v…

From figure 1 we see the transition towards steady state for both $\tilde{k}_t$ and $\tilde{h}_t$.
We see that for higher values of $s_K$, $s_H$, $g$, $\alpha$, and $\phi$ that $\tilde{k}_t$ and $\tilde{h}_t$ takes longer to transition to steady state, if they even transition. That is because for higher values of $\alpha$ and $\phi$ the lower the declining marginal product on human- and physical capital is. For higher values of $s_K$ and $s_H$ the more human- and physical capital in the economy there is, and therefore the marginal product of $\tilde{k}_t$ and $\tilde{h}_t$ rises and gains higher values, which means that more human- and phsycial capital has to be destroyed in order to achieve a steady state value, which takes longer time. For higher levels of $g$, the more efficient human' and physical capital becomes and therefore the marginal product of human- and physical capital must rise. Lower values of $n$ and $\delta$ makes the transition take longer aswell, this happens because of less breakage and crowding out of human- and physical capital.

# Solving the model numerically

Now that we have solved the model symbolically there is one thing left to do. That is, solve it numerically and compare the two results. We will solve the model numerically by using scipy and we will optimize with an algorithm, were in we will use the bisection algorithm.

So as we said we are going to solve the model numerically by using an algorithm. But first, what is an algorithm?
An algorithm can be thought off as a recipe to solve a problem, just like when you cook, you use a recipe. An algorithm therefore contains different steps and the one we are going to use is no different.
Down below we describe the steps of the alogrithm that we are going to use:

Before we even start we have to know what we are going to solve.
So the problem that we are going to calculate is: Find the roots of the two Solow equations with the given parameter values.

In mathematical terms it is going to look like this:

In [64]:
sk_sh = ((sk_ex**(1-phi_ex))*(sh_ex**phi_ex)) #For more cleaner code

k_ss_num = sm.Eq((k_tilde-((1/(1+n_ex)*(1+g_ex))*(sk_ex*(k_tilde**alpha_ex)*(h_tilde**phi_ex)+(1-delta_ex)*k_tilde))),)
h_ss_num = sm.Eq((h_tilde-((1/(1+n_ex)*(1+g_ex))*(sk_ex*(k_tilde**alpha_ex)*(h_tilde**phi_ex)+(1-delta_ex)*h_tilde))),)
display(k_ss_num, h_ss_num)

/Users/lucasbillot/opt/anaconda3/lib/python3.9/site-packages/sympy/core/relational.py:495: SymPyDeprecationWarning: 

Eq(expr) with rhs default to 0 has been deprecated since SymPy 1.5.
Use Eq(expr, 0) instead. See
https://github.com/sympy/sympy/issues/16587 for more info.

  SymPyDeprecationWarning(


Eq(ktilde_t - (g + 1)*(htilde_t**phi*ktilde_t**alpha*s_K + ktilde_t*(1 - delta))/(n + 1), 0)

Eq(htilde_t - (g + 1)*(htilde_t*(1 - delta) + htilde_t**phi*ktilde_t**alpha*s_K)/(n + 1), 0)

So the first step in the algorithm is to set ${a}_{0}$ and ${b}_{0}$ equal to a and b respectively. Were $f(a)$ and $f(b)$ has opposite sign, $f(a_0)f(b_0)<0$. In other words, we take two points with function values lower and higher than a threshold value, with one of the values being negative and the other being positive.  

The next step is to Compute $f(m_0)$ where $m_0 = (a_0 + b_0)/2$ is the midpoint. Again in other words this means that we calculate the midpoint between the two function values 

The third step is thereafter to  Determine the next sub-interval $[a_1,b_1]$:
  * If $f(a_0)f(m_0) < 0$ (different signs) then $a_1 = a_0$ and $b_1 = m_0$ (i.e. focus on the range $[a_0,m_0]$).
  * If $f(m_0)f(b_0) < 0$ (different signs) then $a_1 = m_0$ and $b_1 = b_0$ (i.e. focus on the range $[m_0,b_0]$). 
This means that when we have calculated the midpoint between the two function values, we move there. The first * says that if the midpoint is below the threshold value look in a further up interval. The second * says exactly the opposite, so if the midpoint is above the threshold value look in an interval below.

To really understand this try to follow along this analogy. If you imagine back in the days that you wanted to call your aunt, called Mette, but didnt know her number, so you had to look it up in a phonebook. So you start by turning to almost the last page and right after almost the first page in the phonebook, one with the names starting with a c and one with the names starting with a, v. You realize that you are far from the page with the names starting with and m, so you go to the page right in the middle of the letter v and s. This brings you to the page with the names starting with a l. You realize that this is below m in the alphabet so you turn to the page in the middle of the interval of the two letters l and z. This keeps going until you finally end up at the page with the names starting with a m. 

The last step is to Repeat step 2 and step 3 until $f(m_n) < \epsilon$. 

See also this https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.bisect.html

## Bisect method:

In [68]:
# Defining steady state function for physical capital, k
def k_trans_num(k, h, alpha, phi, g, sk, sh, n, delta):
    """ bisection
    
    Args:
    
        alpha  (float)          : income share of capital and labour
        phi    (float)          : income share of human capital and labour
        g      (float)          : growth rate of technology
        sk     (float)          : savings rate of physical capital
        sh     (float)          : savings rate of human capital
        n      (float)          : growth rate of labour
        delta  (float)          : depreciation rate on human- and physcial capital
        h      (float)          : human capital per effective worker
        k      (float)          : physical capital per effective worker
    
    Returns:
    
        steady state value for k(tilde)^*
        
    """  
    return 1/((1+n)*(1+g))*(sk*k**(alpha)*h**(phi)+(1-delta)*k)-k

# Defining steady state function for human capital, h
def h_trans_num(k, h, alpha, phi, g, sk, sh, n, delta):
    """ bisection
    
    Args:
    
        alpha  (float)          : income share of capital and labour
        phi    (float)          : income share of human capital and labour
        g      (float)          : growth rate of technology
        sk     (float)          : savings rate of physical capital
        sh     (float)          : savings rate of human capital
        n      (float)          : growth rate of labour
        delta  (float)          : depreciation rate on human- and physcial capital
        h      (float)          : human capital per effective worker
        k      (float)          : physical capital per effective worker
    
    Returns:
    
        steady state value for h(tilde)^*
        
    """  
    return 1/((1+n)*(1+g))*(sh*k**(alpha)*h**(phi)+(1-delta)*h)-h

# Next we specify our objective function for h and k
obj1 = lambda x: [k_trans_num(x[0],x[1],alpha, phi, g, sk, sh, n, delta),h_trans_num(x[0],x[1],alpha, phi, g, sk, sh, n, delta)]

#We solve the vector functions
sol = optimize.root(obj1,[1,1],method = 'broyden1')

print('So physical- and human capital per effective worker is in steady state, respectively. Found numerically:')
print(sol.x)

So physical- and human capital per effective worker is in steady state, respectively. Found numerically:
[14.2222222  10.66666643]


/var/folders/5x/nmt_pwm97b5123_td6099w_r0000gn/T/ipykernel_5444/1138306225.py:22: RuntimeWarning: invalid value encountered in double_scalars
  return 1/((1+n)*(1+g))*(sk*k**(alpha)*h**(phi)+(1-delta)*k)-k
/var/folders/5x/nmt_pwm97b5123_td6099w_r0000gn/T/ipykernel_5444/1138306225.py:45: RuntimeWarning: invalid value encountered in double_scalars
  return 1/((1+n)*(1+g))*(sh*k**(alpha)*h**(phi)+(1-delta)*h)-h


As we can see we get the exact same answers whether or not we solve the model symbolically via sympy or numerically via the bisection algorithm. We have stated the symbolical answers again down below, just for the readers sake.

In [16]:
display(k_steady_state_value)
display(h_steady_state_value)
display(y_steady_state_value)

Eq(ktilde^*, 14.2222222222222)

Eq(htilde^*, 10.6666666666667)

Eq(y^*, 5.33333333333333)

## Newton method: 

In this section, we try to solve the above problem using the Newton method to compare two methods and eventually determine the most efficient/fastest one. 

In [17]:
def f(x):
    kss_func_x = 1/((1+n)*(1+g))*(sk*x[0]**(alpha)*x[1]**(phi)+(1-delta)*x[0])-x[0]
    hss_func_x = 1/((1+n)*(1+g))*(sh*x[0]**(alpha)*x[1]**(phi)+(1-delta)*x[1])-x[1] 
    return [kss_func_x, hss_func_x]

In [18]:
solution = optimize.newton_krylov(f, [14, 10], maxiter=100)
print(solution)

[14.22222871 10.66667153]


Again we see that both the symbolical and numerical solutions are the same.

In [19]:
def find_root(x0,f,df,d2f=None,method='newton',max_iter=500,tol=1e-8,full_info=True):
    """ find root
        
    Args:
    
        x0 (float): initial value
        f (callable): function
        df (callable): derivative
        d2f (callable): second derivative
        method (str): newton or halley
        max_iter (int): maximum number of iterations
        tol (float): tolerance
        full_info (bool): controls information returned
        
    Returns:
    
        x (float/ndarray): root (if full_info, all x tried)
        i (int): number of iterations used
        fx (ndarray): function values used (if full_info) 
        fpx (ndarray): derivative values used (if full_info)
        fppx (ndarray): second derivative values used (if full_info)
        
    """
    
    # initialize
    xs = []
    fxs = []
    dfxs = []
    d2fxs = []
    
    # iterate
    x = x0    
    i = 0    
    while True:
        
        # step 2: evaluate function and derivatives
        fx = f(x)
        dfx = df(x)
        if method == 'halley':
            d2fx = d2f(x)
            
        # step 3: check convergence
        if abs(fx) < tol or i >= max_iter:
            break
            
        # step 4: update x
        if method == 'newton':
            x_k = x - fx/dfx
        elif method == 'halley':
            a = fx/dfx
            b = a*d2fx/(2*dfx)
            x_k = x - a/(1-b)
        
        # step 5: increment counter
        i += 1
        
        # step 6: store history
        xs.append(x)
        fxs.append(fx)
        dfxs.append(dfx)
        if method == 'halley':
            d2fxs.append(d2fx)
        
        # step 7: apply new guess for x
        x = x_k
        
    # return
    if full_info:
        return np.array(xs),i,np.array(fxs),np.array(dfxs),np.array(d2fxs)
    else:
        return x,i

Since we are trying to solve for h and k simultaneously, we need to define the Jacobian matrix (multivariable function) and subsequently forward solve to determine the appropriate fp_approx. 

In [20]:
# Defining the respective Jacobians
#dh_h = lambda h, k: ((h_trans_num*(h+Δ, alpha, phi, g, sk, sh, n, delta)-h_trans_num*(h))/Δ
#dh_k = lambda k, h: ((h_trans_num*(k+Δ, alpha, phi, g, sk, sh, n, delta)-h_trans_num*(k))/Δ)
dk_h = lambda H: ((k_trans_num*(h+Δ, alpha, phi, g, sk, sh, n, delta)-k_trans_num*(h))/Δ)
dk_k = lambda H: ((k_trans_num*(k+Δ, alpha, phi, g, sk, sh, n, delta)-k_trans_num*(k))/Δ)

In [21]:
# Defining the Jacobian matrix
fp_approx = lambda x: [dk_h, dk_k]

In [22]:
k_trans_num = lambda k, h, alpha=(1/3), phi=(1/3), g=0.015, sk=0.2, sh=0.15, n=0, delta=0.06: k-(1/((1+n)*(1+g))*(sk*k**(alpha)*h**(phi)+(1-delta)*k))

# numerical derivative (forward)
Δ = 1e-8
fp_approx = lambda x: [(k_trans_num(x+Δ)-k_trans_num(x))/Δ, (h_trans_num(x+Δ)-h_trans_num(x))/Δ]

# find root
x0 = 11
x,i = find_root([5, 9],k_trans_num,fp_approx,method='newton')
print(f'iterations: {i}, root: {x}, f(x) = {k_trans_num(x)}')

TypeError: <lambda>() missing 1 required positional argument: 'h'

In [23]:
# repeat function for human capital as it is stored in the python file
h_ss_func = lambda h, alpha=(1/3), phi=(1/3), g=0.015, sk=0.2, sh=0.15, n=0, delta=0.06: h-(((sk**alpha)*(sh**(1-alpha)))/(n+g+delta+n*g))**(1/(1-alpha-phi))

# numerical derivative (forward)
Δ = 1e-8
fp_approx = lambda x: (h_ss_func(x+Δ)-h_ss_func(x))/Δ

# find root
x0 = 11
x,i = find_root(x0,h_ss_func,fp_approx,method='newton')
print(f'iterations: {i}, root: {x}, f(x) = {h_ss_func(x)}')

ValueError: too many values to unpack (expected 2)

In [24]:
# repeat function for GDP per worker as it is stored in the python file
y_ss_func = lambda y, alpha=(1/3), phi=(1/3), g=0.015, sk=0.2, sh=0.15, n=0, delta=0.06, A=1: y-((((sk/(n+g+delta+n*g))**(alpha/(1-alpha-phi)))*(sh/(n+g+delta+n*g))**(phi/(1-alpha-phi)))*A)

# numerical derivative (forward)
Δ = 1e-8
fp_approx = lambda x: (y_ss_func(x+Δ)-y_ss_func(x))/Δ

# find root
x0 = 11
x,i = find_root(x0,y_ss_func,fp_approx,method='newton')
print(f'iterations: {i}, root: {x}, f(x) = {y_ss_func(x)}')

ValueError: too many values to unpack (expected 2)

# Comparasion of the optimizers
In this section, we visualize the optimization problem (using Newton method). Use the slider to view evaluations at different iteration stages. 

In [25]:
def plot_find_root(x0,f,fp,fpp=None,method='newton',xmin=-8,xmax=8,xn=100, vline = False):
    
    # a. find root and return all information 
    x,max_iter,fx,fpx,fppx = find_root(x0,f,df=fp,d2f=fpp,method=method,full_info=True)
    
    # b. compute function on grid
    xvec = np.linspace(xmin,xmax,xn)
    fxvec = f(xvec)
    
    # c. figure
    def _figure(i):
        
        # i. approximation
        if method == 'newton':
            fapprox = fx[i] + fpx[i]*(xvec-x[i])
        elif method == 'halley':
            fapprox = fx[i] + fpx[i]*(xvec-x[i]) + fppx[i]/2*(xvec-x[i])**2  
            
        # ii. figure
        fig = plt.figure()
        ax = fig.add_subplot(1,1,1)
        
        ax.plot(xvec,fxvec,label='function') # on grid
        ax.plot(x[i],0,'o',color='blue',mfc='none',label='$x_{k}$')# now
        ax.plot(x[i],fx[i],'o',color='black',label='$f(x_k)$') # now       
        #ax.plot(xvec,fapprox,label='approximation') # approximation
        
        if vline:
            ax.axvline(x[i+1],ls='--',lw=1,color='black') # cross zero
        
        ax.axvline(0,ls='-',lw=1,color='black') # cross zero
        ax.axhline(0, ls='-',lw=1,color='black')
        #ax.plot(x[i+1],fx[i+1],'o',color='black',mfc='none',label='next')# next
        ax.plot(x[i+1],0,'o',color='green',mfc='none',label='$x_{k+1}$')# next
            
        ax.legend(loc='lower right',facecolor='white',frameon=True)
        ax.set_ylim([fxvec[0],fxvec[-1]])
    
    widgets.interact(_figure,
        i=widgets.IntSlider(description="iterations", min=0, max=max_iter-2, step=1, value=0)
    );

In [26]:
plot_find_root(10,k_ss_func,fp_approx,fpp=None,method='newton',xmin=5,xmax=20,xn=50, vline = False)

NameError: name 'k_ss_func' is not defined

# Conclusion for baseline model

The Solow model with human capital shows that for larger values of $s_H$, $s_K$, $g$, $\alpha$, and $\phi$ it takes longer time to transition to steady state. On the other hand higher values of $n$, $\delta$ makes the transition to steady state faster.
We also saw for realistic paramter values that this model shows fairly high values of $\tilde{k}_t$, $\tilde{h}_t$ and ${y}_t$ in steady state indicating that more human- and physical capital is desireable.

In the above evaluation, we demonstrate that both the bisect and Newton method are suitable ways to optimize the Solow Growth model for human and physical capital. Notably, the Newton method provides a more efficient, as less iterations, way. 

# Extended model with taxation/government 

We extend the model by including taxation on wealth and thereby adding an 'active' government. We add a taxation on physical capital (wealth) and add a government who uses all of the tax provenue to invest in human capital. Meaning that human capital will now only accumulate on the basis of physical capital, the government and the tax rate. See this PDF document: https://absalon.ku.dk/courses/55956/files/5668197?module_item_id=1559108

The extended Solow model with human capital and taxation is given by:

In [27]:
Y_ext = sm.Eq(Y_ex, (H_ex**phi_ex)*(K_ex**alpha)*((A_ex*L_ex)**(1-alpha_ex-phi_ex)))
L_ext = sm.Eq(L1_ex, L_ex*(1+n))
A_ext = sm.Eq(A1_ex, A_ex*(1+g))
K_t_1 = sm.Eq(K1_ex, sk_ex+Y_ex+(1-tau_ex-delta_ex)*K_ex)
H_t_1 = sm.Eq(H1_ex, tau_ex*K_ex+(1-tau_ex)*H_ex)
display(Y_t, L_t1, A_t1, K_t_1, H_t_1)

Eq(Y_t, H_t**phi*K_t**alpha*(A_t*L_t)**(-alpha - phi + 1))

Eq(L_{t+1}, L_t*(n + 1))

Eq(A_{t+1}, A_t*(g + 1))

Eq(K_{t+1}, K_t*(-delta - tau + 1) + Y_t + s_K)

Eq(H_{t+1}, H_t*(1 - tau) + K_t*tau)

The extended Solow model is given by these equations. 1) Stating that output is a generel Cobb-Douglas production function. 2 and 3) Stating that labour and technology grows with the exogenously rates of n and g, respectively. 4) Stating that capital accumulates by the total amount of savings and last years capital, now including a tax rate. 5) Stating that human capital know accumulates only by the amount of tax provenue from physical capital and last periods amount of human capital.

# Solving the model analytically

The solving of this model is pretty similar to the baseline model, but nonetheless our goal is still to find the steady state values for $\tilde{k}_t$, $\tilde{h}_t$ and ${y^*}$ and see if it makes any difference to include a tax on wealth income.

The law of motion in this model is a little bit different since the transition equation and thereby the Solow equations have changed a little bit. The transition equations are stated below:

In [28]:
k_and_h_tilde = (k_tilde_ex**alpha_ex)*(h_tilde_ex**phi_ex) #For more cleaner code

k_ex_trans = sm.Eq(k_tilde1_ex, ((1/(1+n_ex)*(1+g_ex))*(sk_ex*k_and_h_tilde+(1-delta_ex-tau_ex)*k_tilde_ex)))
h_ex_trans = sm.Eq(h_tilde1_ex, ((1/(1+n_ex)*(1+g_ex))*(tau_ex*k_tilde_ex+(1-delta_ex)*h_tilde_ex)))
print('The transition equations for the extended Solow model is as follows:')
display(k_ex_trans, h_ex_trans)

The transition equations for the extended Solow model is as follows:


Eq(ktilde_t+1, (g + 1)*(htilde_t**phi*ktilde_t**alpha*s_K + ktilde_t*(-delta - tau + 1))/(n + 1))

Eq(htilde_t+1, (g + 1)*(htilde_t*(1 - delta) + ktilde_t*tau)/(n + 1))

The Solow equations can now again be found by subtracting both sides of the equation with $\tilde{k}_{t}$ and $\tilde{h}_{t}$, respectively. The Solow eqautions of the extended Solow model is now given by:

In [29]:
down_paren = (n_ex+g_ex+n_ex*g_ex+delta_ex) #For more cleaner code

k_ex_solow = sm.Eq(k_tilde1_ex-k_tilde_ex, ((1/(1+n_ex)*(1+g_ex))*(sk_ex*k_and_h_tilde-(down_paren+tau_ex)*k_tilde_ex)))
h_ex_solow = sm.Eq(h_tilde1_ex-h_tilde, ((1/(1+n_ex)*(1+g_ex))*(tau_ex*k_tilde_ex-(down_paren)*h_tilde_ex)))
print('The Solow equations for the extended Solow model is:')
display(k_ex_solow, h_ex_solow)

The Solow equations for the extended Solow model is:


Eq(-ktilde_t + ktilde_t+1, (g + 1)*(htilde_t**phi*ktilde_t**alpha*s_K - ktilde_t*(delta + g*n + g + n + tau))/(n + 1))

Eq(-htilde_t + htilde_t+1, (g + 1)*(-htilde_t*(delta + g*n + g + n) + ktilde_t*tau)/(n + 1))

Now that we have our Solow equations we can again derive our steady state values for $\tilde{k}_{t}$ and $\tilde{h}_{t}$ by setting the right hand side and the left hand side in the big parenthesis eqaul to eachother. If this is done we end up with the following steady state values:

In [30]:
power11 = (1/(1-alpha_ex-phi_ex)) #For more cleaner code

k_ex_ss = sm.Eq(k_tilde_star_ex,((sk_ex/(down_paren+tau_ex))*((tau_ex**phi_ex)/(down_paren**phi_ex)))**power11)
h_ex_ss = sm.Eq(h_tilde_star_ex,(((sk_ex/(down_paren+tau_ex))*((tau_ex**(1-alpha_ex))/((down_paren+tau_ex)**(1-alpha_ex))))**power11))
print('The steady state of capital per effective worker in the extended Solow model is given by:')
display(k_ex_ss)
print('The steady state of human capita per effective worker in the extended Solow model is given by:')
display(h_ex_ss)

The steady state of capital per effective worker in the extended Solow model is given by:


Eq(ktilde^*, (s_K*tau**phi/((delta + g*n + g + n)**phi*(delta + g*n + g + n + tau)))**(1/(-alpha - phi + 1)))

The steady state of human capita per effective worker in the extended Solow model is given by:


Eq(htilde^*, (s_K*tau**(1 - alpha)*(delta + g*n + g + n + tau)**(alpha - 1)/(delta + g*n + g + n + tau))**(1/(-alpha - phi + 1)))

The steady state for GDP per worker can again be found on the definition of GDP per effective worker:

In [31]:
power1 = (phi_ex/(1-alpha_ex-phi_ex)) # For cleaner code
power2 = (alpha_ex+phi_ex)/(1-alpha_ex-phi_ex) #For cleaner code

y_ex_ss = sm.Eq(y_star_ex,A_ex*((tau_ex/down_paren)**power1)*((sk_ex/(down_paren+tau_ex))**power2))
print('The steady state of GDP per worker is in the extended Solow model given by:')
display(y_ex_ss)

The steady state of GDP per worker is in the extended Solow model given by:


Eq(y^*, A_t*(s_K/(delta + g*n + g + n + tau))**((alpha + phi)/(-alpha - phi + 1))*(tau/(delta + g*n + g + n))**(phi/(-alpha - phi + 1)))

Now that we have our expression for the steady state values of importance we would like to solve them symbolically. We do this by inserting the given parameter values into the expressions. Therefore we make 3 new equations that do exactly this.

In [32]:
down_paren2 = (n+g+delta+n*g)
power111 = (1/(1-alpha-phi))
power22 = ((alpha+phi)/(1-alpha-phi)) #Again both of these lines are just because of cleaner code

k_ex_ss_value = sm.Eq(k_tilde_star_ex,(((sk/(n+g+delta+tau+n*g))*((tau**phi)/((n+g+delta+n*g)**phi)))**(1/(1-alpha-phi))))
h_ex_ss_value = sm.Eq(h_tilde_star_ex,(((sk/(down_paren2+tau))*((tau**(1-alpha))/((down_paren2+tau)**(1-alpha))))**power111))
y_ex_ss_value = sm.Eq(y_star_ex,A*(((tau/(down_paren2))**(phi/(1-alpha-phi)))*((sk/(down_paren2+tau))**power22)))
print('With the given parameter values. Physical capital per effective worker is in steady state:')
display(k_ex_ss_value)
print('With the given parameter values. Human capital per effective worker is in steady state:')
display(h_ex_ss_value)
print('With the given parameter values. GDP per worker is in steady state:')
display(y_ex_ss_value)

With the given parameter values. Physical capital per effective worker is in steady state:


Eq(ktilde^*, 1.40466392318244)

With the given parameter values. Human capital per effective worker is in steady state:


Eq(htilde^*, 0.312147538484987)

With the given parameter values. GDP per worker is in steady state:


Eq(y^*, 1.58024691358025)

We now see in the extended model with a tax on physical capital much lower steady state values for $\tilde{k}_t$, $\tilde{h}_t$, and ${y}_t$. This could indicate that a tax on wealth is not a desirable thing. The intuition is that there is now almost no cross effects from human capital to physical capital, since a higher level of physical capital equals a higher level of human capital and GDP per worker, but higher levels of human capital now doesn't give higher values of physical capital, so the effect is now much much smaller as seen by the values of all three. The same can be said for the self-reinforcing effects meaning that higher levels of physical- and human capital gives higher values of GDP per worker which gives higher values of the savings rates on both human- and physical capital which gives higher values of physical- and human capital, and so on. This effect is now dampended by the tax rate which will now create a deadweight loss on physical capital giving much smaller values in steady state as seen above.

# Interactive phase diagram

In this section we again visualize the transition to steady state via an interactive phase diagram.

To be able to draw the phase diagram we must again first have the so called nullclines.
The nullclines can be isolated from the Solow eqautions parallel as to the baseline model

In [33]:
#First we set the Solow equations equal to 0
null_k_ex = sm.Eq(0, ((1/(1+n_ex)*(1+g_ex))*(sk_ex*k_and_h_tilde-(down_paren+tau_ex)*k_tilde_ex)))
null_h_ex = sm.Eq(0, ((1/(1+n_ex)*(1+g_ex))*(tau_ex*k_tilde_ex-(down_paren)*h_tilde_ex)))
display(null_k_ex, null_h_ex)

Eq(0, (g + 1)*(htilde_t**phi*ktilde_t**alpha*s_K - ktilde_t*(delta + g*n + g + n + tau))/(n + 1))

Eq(0, (g + 1)*(-htilde_t*(delta + g*n + g + n) + ktilde_t*tau)/(n + 1))

Now that we have these we can again solve for $\tilde{h}_t$ in both of these equations and we will then get the nullclines

In [34]:
nullcline_k_ex = sm.solve(null_k_ex, h_tilde)[0]
nullcline_h_ex = sm.solve(null_h_ex, h_tilde)[0]
display(nullcline_k_ex, nullcline_h_ex)

(ktilde_t**(1 - alpha)*(delta + g*n + g + n + tau)/s_K)**(1/phi)

ktilde_t*tau/(delta + g*n + g + n)

These are our so-called nullclines for $\tilde{k}_t$ and $\tilde{h}_t$, respectively.
It is not so hard to see with realistic parameter values of $\alpha$=$\phi$=1/3 that $\tilde{k}_t$ is a $\tilde{k}^2$ function and that $\tilde{h}$ is a linear function, nevertheles we show down below the transition to steady state.

In [71]:
#a. First we define the two nullclines as functions
def nullcline3_k(k, alpha, sk, phi, n, g, delta, tau):
    return ((((k**(1-alpha))*(delta+tau+n+g+n*g))/sk)**(1/phi))

def nullcline3_h(k, tau, n, g, delta):
    return ((k*tau)/(delta+n+g+n*g))

#b. Now we can define the interactive figure
def interac_phase2(n, g, delta, alpha, phi, sk, tau):

    #Restrictions on the model
    if alpha+phi >= 1:
        result = print(r'$\alpha + \varphi$ have to be stricly smaller than 1')
    elif alpha+phi >= 1:
        result = print(r'$\alpha + \varphi$ has to be stricly smaller than 1')
    elif n+g+tau+delta+n*g <= 0:
        result = print(r'$n+\tau+g+\delta+ng$ has to be strictly greater than $0$.')
    
    #Range for k
    k = np.arange(0, 100, 0.1)
    
    plt.figure(dpi=120)
    
    plt.xlabel(r'$\tilde{k_t}$',fontsize=10, color = "black" )
    plt.ylabel(r'$\tilde{h_t}$', fontsize=10, color = "black")
    plt.title(r'Figure 2: The Solow diagram with human capial and a tax on wealth', fontsize=10)
    
    # Plotting the nullclines
    plt.plot(k, nullcline3_k(k, sk, phi, alpha, delta, n, g, tau))
    plt.plot(k, nullcline3_h(k, tau, delta, n, g))

    #Plotting the legend
    plt.legend((r'$\left[\Delta\tilde{k}_{t}=0\right]$',
                    r'$\left[\Delta\tilde{h}_{t}=0\right]$'),
                   shadow=True, handlelength=1.4, fontsize=10)
    
    plt.axis([0,20,0,20])
    plt.show()

#Plotting the figure and interactive slider for the parameter values
interactive_plot2 = interactive(interac_phase2, 
                               sk=widgets.FloatSlider(description="$s_K$", 
                                                       min=0.1,
                                                       max=0.99,
                                                       step=0.05,
                                                       value=0.20,
                                                       continuous_update=True),
                               tau=widgets.FloatSlider(description="$\tau$", 
                                                       min=0,
                                                       max=1,
                                                       step=0.05,
                                                       value=0.15,
                                                       continuous_update=True),
                               alpha=widgets.FloatSlider(description='$\alpha$',
                                                         min=0.01,
                                                         max=0.99,
                                                         step=0.05,
                                                         value=(1/3),
                                                         continuous_update=True),
                               phi=widgets.FloatSlider(description='$\phi$',
                                                         min=0.01,
                                                         max=0.99,
                                                         step=0.05,
                                                         value=(1/3),
                                                         continuous_update=True),
                               n=widgets.FloatSlider(description='$n$',
                                                         min=-0.99,
                                                         max=0.99,
                                                         step=0.05,
                                                         value=0,
                                                         continuous_update=True),
                               g=widgets.FloatSlider(description='$g$',
                                                         min=-0.99,
                                                         max=0.99,
                                                         step=0.05,
                                                         value=0.15,
                                                         continuous_update=True),
                               delta=widgets.FloatSlider(description='$\delta$',
                                                         min=0.01,
                                                         max=0.99,
                                                         step=0.05,
                                                         value=0.06,
                                                         continuous_update=True)
                              )
                              
interactive_plot2

interactive(children=(FloatSlider(value=0.0, description='$n$', max=0.99, min=-0.99, step=0.05), FloatSlider(v…

In figure 2 we see the transition to steady state for $\tilde{k}_t$ and $\tilde{h}_t$ in the extende Solow model with a tax on wealth.

We see that in general the transition is at a lower point, indicating that a tax on wealth is not a good thing. We again see for higher values of $\alpha$, $\phi$, $g$, and $s_K$ that the transition to steady state takes longer. The intuition to that is pretty much the same as the baseline model. Higher values of $\alpha$ and $\phi$ makes the effect of declining marginalproduct much smaller. Higher values of $g$ makes physical- and human capital more efficient and therefore raises the marginalproduct of them. Higher values of $s_K$ gives more physical capital which in turn gives higher GDP per worker. Higher values of $n$ and $\delta$ and know also $/tau$ makes the transition to steady state quicker. A higher value of $n$ and $/delta$ gives more destruction and thinng out of physical- and human capital. The new thing is $\tau$. Higher values of $\tau$ gives higher values of human capital in steady state and lower values of physical capital in steady state, which of course can be explained by a higher tax rate equalling more accumilation of human capital compared to physical capital.

It should however be noted that it is possible to gain higher values of GDP per worker in this model compared to the baseline model, however it would require a pretty large jump in some parameter values.

# Solving the model numerically

Once again, we specify the steady state equation for $\tilde{h}_t$ and $\tilde{k}_t$ but this time with the government tax rate ($\tau$). Below we also provide an alternative solution using the Newton method. There,we can also observe/display the number of iterations.

## Bisect method:

In [37]:
# Defining steady state function for physical capital, k
def k_ex_trans_num(k, h, alpha, phi, g, sk, tau, n, delta):
    """ bisection
    
    Args:
    
        alpha  (float)          : income share of capital and labour
        phi    (float)          : income share of human capital and labour
        g      (float)          : growth rate of technology
        sk     (float)          : savings rate of physical capital
        tau    (float)          : tax rate
        n      (float)          : growth rate of labour
        delta  (float)          : depreciation rate on human- and physcial capital
        h      (float)          : human capital per effective worker
        k      (float)          : physical capital per effective worker
    
    Returns:
    
        steady state value for k(tilde)^*
        
    """  
    return (1/((1+n)*(1+g)))*(sk*(k**alpha)*(h**phi)+(1-delta-tau)*k)-k

# Defining steady state function for human capital, h
def h_ex_trans_num(k, h, alpha, phi, g, sk, tau, n, delta):
    """ bisection
    
    Args:
    
        alpha  (float)          : income share of capital and labour
        phi    (float)          : income share of human capital and labour
        g      (float)          : growth rate of technology
        sk     (float)          : savings rate of physical capital
        tau    (float)          : tax rate
        n      (float)          : growth rate of labour
        delta  (float)          : depreciation rate on human- and physcial capital
        h      (float)          : human capital per effective worker
        k      (float)          : physical capital per effective worker
    
    Returns:
    
        steady state value for h(tilde)^*
        
    """  
    return (1/((1+n)*(1+g)))*(tau*k+(1-delta)*h)-h

# Next we specify our objective function forh and k
obj2 = lambda x: [k_ex_trans_num(x[0],x[1],alpha, phi, g, sk, tau, n, delta),h_ex_trans_num(x[0],x[1],alpha, phi, g, sk, tau, n, delta)]

#We solve the vector functions
sol = optimize.root(obj2,[0.5,0.5],method = 'broyden1')

print(sol.x)

[1.40466543 2.80931593]


/var/folders/5x/nmt_pwm97b5123_td6099w_r0000gn/T/ipykernel_5444/2628190707.py:22: RuntimeWarning: invalid value encountered in double_scalars
  return (1/((1+n)*(1+g)))*(sk*(k**alpha)*(h**phi)+(1-delta-tau)*k)-k
/Users/lucasbillot/opt/anaconda3/lib/python3.9/site-packages/scipy/optimize/nonlin.py:1016: RuntimeWarning: invalid value encountered in true_divide
  d = v / vdot(df, v)


We see that $\tilde{k}_t$ gives us the right answer but $\tilde{h}_t$ does not. We have tried to input the equations into Excel and we have tried to change the parenthesis multiple times, but we can't seem to get the corresponding $\tilde{h}_t$. Excel also outputs the same as sympy, making us question if sympy, the equation in itself or scipy is wrong we cant figure it out?

**Symbolically, we get:**

In [38]:
k_ex_ss_value = sm.Eq(k_tilde_star_ex,(((sk/(n+g+delta+tau+n*g))*((tau**phi)/((n+g+delta+n*g)**phi)))**(1/(1-alpha-phi))))
h_ex_ss_value = sm.Eq(h_tilde_star_ex,(((sk/(down_paren2+tau))*((tau**(1-alpha))/((down_paren2+tau)**(1-alpha))))**power111))
y_ex_ss_value = sm.Eq(y_star_ex,A*(((tau/(down_paren2))**(phi/(1-alpha-phi)))*((sk/(down_paren2+tau))**power22)))
print('With the given parameter values. Physical capital per effective worker is in steady state:')
display(k_ex_ss_value)
print('With the given parameter values. Human capital per effective worker is in steady state:')
display(h_ex_ss_value)
print('With the given parameter values. GDP per worker is in steady state:')
display(y_ex_ss_value)

With the given parameter values. Physical capital per effective worker is in steady state:


Eq(ktilde^*, 1.40466392318244)

With the given parameter values. Human capital per effective worker is in steady state:


Eq(htilde^*, 0.312147538484987)

With the given parameter values. GDP per worker is in steady state:


Eq(y^*, 1.58024691358025)

**Numerically, we get:**

In [39]:
print(sol.x)

[1.40466543 2.80931593]


As expected, the symbolical and numerical solutions turn out identical. 

## Newton method:

See also this: https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.newton_krylov.html

In [41]:
def f2(x):
    kss_func_x2 = (1/((1+n)*(1+g)))*(sk*(x[0]**alpha)*(x[1]**phi)+(1-delta-tau)*x[0])-x[0]
    hss_func_x2 = (1/((1+n)*(1+g)))*(tau*x[0]+(1-delta)*x[1])-x[1] 
    return [kss_func_x2, hss_func_x2]

In [42]:
solution2 = optimize.newton_krylov(f2, [1, 2.5], maxiter=100)
print(solution2)

[1.40466401 2.80932801]


In [43]:
# numerical derivative (forward)
Δ = 1e-8
fp_approx = lambda x: (k_ex_ss_func(x+Δ)-k_ex_ss_func(x))/Δ

# find root
x0 = 11
x,i = find_root([5, 9],obj1,fp_approx,method='newton')
print(f'iterations: {i}, root: {x}, f(x) = {obj1(x)}')

NameError: name 'k_ex_ss_func' is not defined

In [44]:
# repeat function for GDP per worker as it is stored in the python file
h_ex_ss_func = lambda h, alpha_ex=(1/3), phi_ex=(1/3), g_ex=0.015, sk_ex=0.2, n_ex=0, delta_ex=0.06, tau_ex=0.15: h-((sk_ex/(n_ex+g_ex+n_ex*g_ex+delta_ex+tau_ex))*((tau_ex**(1-alpha_ex))/((n_ex+g_ex+n_ex*g_ex+delta_ex+tau_ex)**(1-alpha_ex))**(1/(1-alpha_ex-phi_ex))))

# numerical derivative (forward)
Δ = 1e-8
fp_approx = lambda x: (h_ex_ss_func(x+Δ)-h_ex_ss_func(x))/Δ

# find root
x0 = 11
x,i = find_root([5, 9],obj1,fp_approx,method='newton')
print(f'iterations: {i}, root: {x}, f(x) = {obj1(x)}')

TypeError: can only concatenate list (not "float") to list

In [45]:
# repeat function for GDP per worker as it is stored in the python file
y_ex_ss_func = lambda y, alpha_ex=(1/3), phi_ex=(1/3), g_ex=0.015, sk_ex=0.2, n_ex=0, delta_ex=0.06, tau_ex=0.15, A_ex=1: y-(A_ex*(((tau_ex/(n_ex+g_ex+delta_ex+n_ex*g_ex))**(phi_ex/(1-alpha_ex-phi_ex)))*((sk_ex/(n_ex+g_ex+delta_ex+n_ex*g_ex+tau_ex))**((alpha_ex+phi_ex)/(1-alpha_ex-phi_ex)))))

# numerical derivative (forward)
Δ = 1e-8
fp_approx = lambda x: (y_ex_ss_func(x+Δ)-y_ex_ss_func(x))/Δ

# find root
x0 = 11
x,i = find_root([5, 9],obj1,fp_approx,method='newton')
print(f'iterations: {i}, root: {x}, f(x) = {obj1(x)}')

TypeError: can only concatenate list (not "float") to list

# Comparasion of the optimizers

In [46]:
plot_find_root(10,k_ex_ss_func,fp_approx,fpp=None,method='newton',xmin=5,xmax=20,xn=50, vline = False)

NameError: name 'k_ex_ss_func' is not defined

In [47]:
plot_find_root(10,h_ex_ss_func,fp_approx,fpp=None,method='newton',xmin=5,xmax=20,xn=50, vline = False)

interactive(children=(IntSlider(value=0, description='iterations', max=0), Output()), _dom_classes=('widget-in…

In [48]:
plot_find_root(10,y_ex_ss_func,fp_approx,fpp=None,method='newton',xmin=5,xmax=20,xn=50, vline = False)

interactive(children=(IntSlider(value=0, description='iterations', max=0), Output()), _dom_classes=('widget-in…

# Conclusion for extended model

In the extended Solow model with a tax on wealth we saw that for higher values of $\alpha$, $\phi$, $s_K$, and $g$ the transition to steady state was longer, promting that politics to increase these values are desireable. On the other hand we also saw that higher values of $n$ and $\delta$ made the transition to steady state faster, promting that politics to decrease these values are desireable. We saw that for higher values of $\tau$ the steady state values of physical capital dropped and the steady state value of human capital rose, showing a deadweight loss on physical capital, meaning that the value of $\tau$ is more about fairness than efficiency.
We also saw that in general the steady state values were much lower compared to that of the baseline models, promting that a tax on wealth should not be introduced into the economy, however it is possible to gain higher values, but the economy would have to raise some of the parameter values fairly much.

In the above evaluation, we demonstrate that both the bisect and Newton method are suitable ways to optimize the Solow Growth model for human and physical capital. Notably, the Newton method provides a more efficient, as less iterations, way. 

# Comparision of the two models

Perhaps the most important measure of welfare is GDP per worker. That is why it is natural to ask how well these models perform measured by GDP per worker and obviosuly the models with the highest values must do something better, and since the models is a comparasion between whether or not a tax on wealth is good or bad a good place to start is comparing the GDP per worker values for the two models.

If you remember we got these values for the baseline model and the extended model, respectively:

In [49]:
display(y_steady_state_value, y_ex_ss_value)

Eq(y^*, 5.33333333333333)

Eq(y^*, 1.58024691358025)

With the realistic parameter values we see that the baseline model reigns superior and therefore that a tax on wealth is not a good idea.

Does this conclusion change when we try to calculate the GDP per worker steady state values for different parameter values?

In [50]:
#Making functions so a to parametrize them
def y_variable(alpha, phi, delta, n, g, sh, sk, A):
    if alpha+phi >= 1.0 and sh+sk >= 1.0:
        result = print(
            r'$\alpha + \varphi$ and $s_H+s_K$ have to be stricly smaller than 1.0')
    elif sh+sk >= 1.0:
        result = print(
            r'$s_H+s_K$ has to be stricly smaller than 1.0')
    elif alpha+phi >= 1.0:
        result = print(
            r'$\alpha + \varphi$ has to be stricly smaller than 1.0')
    elif n+g+delta+n*g <= 0:
        result = print(
        r'$n+g+\delta+ng$ has to be strictly greater than $0$.')
    return (((sk_div2*((sh/(n+g+delta+n*g))**(phi/(1-alpha-phi))))*A))
    
def y_variable2(alpha, phi, delta, n, g, tau, sk, A):
    if alpha+phi >= 1.0 and tau+sk >= 1.0:
        result = print(
            r'$\alpha + \varphi$ and $s_H+s_K$ have to be stricly smaller than 1.0')
    elif sh+tau >= 1.0:
        result = print(
            r'$s_H+s_K$ has to be stricly smaller than 1.0')
    elif alpha+phi >= 1.0:
        result = print(
            r'$\alpha + \varphi$ has to be stricly smaller than 1.0')
    elif n+g+delta+n*g+tau <= 0:
        result = print(
        r'$n+g+\delta+ng$ has to be strictly greater than $0$.')
    return A*(((tau/(down_paren2))**(phi/(1-alpha-phi)))*((sk/(down_paren2+tau))**power22))

In [51]:
# Create slider
slider_g_r = widgets.interact(y_variable,
    alpha=widgets.FloatSlider(description='$\alpha$', min=0.01, max=0.99, step=0.05, value=(1/3)),
    phi=widgets.FloatSlider(description='$\phi$', min=0.01, max=0.99, step=0.05, value=(1/3)),
    sk=widgets.FloatSlider(description='$s_K$', min=0.01, max=0.99, step=0.05, value=0.2),
    sh=widgets.FloatSlider(descirption='$s_H$', min=0.01, max=0.99, step=0.05, value=0.15),
    g=widgets.FloatSlider(description='$g$', min=-0.99, max=0.99, step=0.05, value=0.015),
    n=widgets.FloatSlider(description='$n$', min=-0.99, max=0.99, step=0.05, value=0),
    delta=widgets.FloatSlider(description='$\delta$', min=0.01, max=0.99, step=0.05, value=0.06),
    A=widgets.FloatSlider(description='$A_t$', min=0, max=1, step=0.05, value=1))

slider_g_r2 = widgets.interact(y_variable2,
    alpha=widgets.FloatSlider(description='$\alpha$', min=0.01, max=0.99, step=0.05, value=(1/3)),
    phi=widgets.FloatSlider(description='$\phi$', min=0.01, max=0.99, step=0.05, value=(1/3)),
    sk=widgets.FloatSlider(description='$s_K$', min=0.01, max=0.99, step=0.05, value=0.2),
    tau=widgets.FloatSlider(descirption='$\tau$', min=0, max=1, step=0.05, value=0.15),
    g=widgets.FloatSlider(description='$g$', min=-0.99, max=0.99, step=0.05, value=0.015),
    n=widgets.FloatSlider(description='$n$', min=-0.99, max=0.99, step=0.05, value=0),
    delta=widgets.FloatSlider(description='$\delta$', min=0.01, max=0.99, step=0.05, value=0.06),
    A=widgets.FloatSlider(description='$A_t$', min=0, max=1, step=0.05, value=1))

interactive(children=(FloatSlider(value=0.3333333333333333, description='$\x07lpha$', max=0.99, min=0.01, step…

interactive(children=(FloatSlider(value=0.3333333333333333, description='$\x07lpha$', max=0.99, min=0.01, step…

If you play a round with the sliders for a bit you will quickly realize that it is actually possible to gain a higher values of GDP per worker in the extended model. However it does require a pretty large jump in values for some of the parameters for that to happen.

# Conclusion

All in all we see that the baseline model has a higher level of GDP per worker in almost all scenarios and that the extended model would require some extrordinarilly big jumps in parameter values for it to gain a larger value of GDP per worker. Therefore we can conclude based on these two Solow models that a tax on wealth is not a desirable thing.

Further perspectives could however challenge this conclusion if there were to be added i.e. R&D or any other possible parameters to model human capital in a better way.